In [ ]:
import os
import datetime
import pandas as pd
import json
import pprint
from collections import OrderedDict
import re

#listim = ['immigra', 'refugee', 'migra']
#immigration_patt = re.compile(pattern=r"\b("+"|".join(listim)+")", flags=re.I)

def remove_extra_spaces(s):
    while '  ' in s:
        s = s.replace('  ',' ')
    return s.strip()

def remove_punct(s):
    s = s.replace(".", " ")
    s = s.replace(",", " ")
    s = s.replace("'", "")
    s = s.replace('"', " ")
    s = s.replace('-', "")
    s = s.replace('`', "")
    s = s.replace(';', " ")
    s = s.replace('*', " ")
    s = s.replace('~', " ")
    s = s.replace('_', " ")
    s = s.replace('\\', "")
    s = s.replace('•', " ")
    
    return s.strip()

def stripspeech(s):
    s = s.replace('§', 'sec')
    s = remove_extra_spaces(remove_punct(s).lower())
    s = s.replace(' ', '')
    s = s.replace('9111', '911')
    s = s.replace('9/11', '911')
    return s


In [ ]:
df=pd.read_hdf('./crec2015to2021.hdf')

In [ ]:
set(df[df.speech.str.lower().str.contains('the house stands adjourned')].speaker)

In [ ]:
# Compare to old data

#valid_speakers = ['mr. ','the ','ms. ','mrs.', 'dr. ','miss']
valid_speakers = ['mr. ','ms. ','mrs.', 'dr. ','miss'] # Drop "The speaker", "The clerk", etc

df_old=pd.read_csv('../created_data/daily_114.csv',sep='\t', index_col='speech_id')
df_old=df_old.loc[df_old.chamber.isin(['H','S'])]

df_old=df_old[df_old.speaker.str.lower().str[0:4].isin(valid_speakers)]


# Check that we are not missing speeches in new data
def short_filter(s):
    return len(stripspeech(s))>=1500

for unique_date in sorted(set(df_old.date)):
    a = df_old[df_old.date==unique_date]
    a = a[a.speech.apply(short_filter)]
    b = df[df.date==unique_date]
    b = b[b.speech.apply(short_filter)]
    
    if len(a) > len(b): # on this day, there are more speeches in old dataset than new dataset
        print(unique_date, len(a), len(b))
        if False and unique_date == '2015-04-17':
            print(a)
            print(a.iloc[0].speech)
            

In [ ]:
import edlib

def match_speeches(a,b):
    match1 = { rndx1 : None for rndx1, r1 in a.iterrows() }
    match2 = { rndx2 : None for rndx2, r2 in b.iterrows() }

    def dfunc(r1,r2):
        return edlib.align(stripspeech(r1.speech), stripspeech(r2.speech))['editDistance']

    for rndx1, r1 in a.iterrows():
        for rndx2, r2 in b.iterrows():
            d=dfunc(r1,r2)
            if d<20:
                match1[rndx1]=rndx2
                match2[rndx2]=rndx1
                break

    for rndx1, r1 in a.iterrows():
        if match1[rndx1] is None:
            print('No match to speech in old dataset')
            print(r1)
            print()

    for rndx2, r2 in b.iterrows():
        if match2[rndx2] is None:
            print('No match to speech in new dataset')
            print(r2)
            print()
